### Iterative point method used to find the optimal strategy in some probabilistic game

The problem is inspired by some Project Euler problem (not pointed out which exactly, to avoid spoilers :)).

Let us assume we are taking part in the following game:

$2N$ cards, labelled from $1$ to $2N$ are placed on the table, faced up.

A turn in this game consists in rolling two $N$-sided, fair dices - with numbers from $1$ to $N$ on each dice.

Assuming dices showed $x$ and $y$ numbers, you are to choose one of three possible moves:
    
1) to turn over the card $x$;

2) to turn over the card $y$;

3) to turn over the card $x+y$.

Note that $x$ may be equal to $y$. Turning over a card consists in turning the card to faced up, being already faced down and vice versa.

The procedure is repeated till you manage to have all $2N$ cards faced down at the same time - then you win.

Compute the expected number of moves that need to be done to win this game, assuming that you always follow the optimal strategy - in terms of minimizing a number of moves. Give your number rounded to $6$ decimal places.

#### Solution

Let us start from some simple notation. Given $2N$ cards, we can express any state of cards faced up/down as some non-negative integer, in the binary system. 

For instance, suppose $N=2$ and we want to express state with cards $1$ and $2$ faced down, and $3$ and $4$ faced up. If we denote cards being faced down by ones, and faced up by zeros, we will get: $0011_b=3$. Basically, state of a $n$-th card is specified by the $n$-th bit from the right in our number ($1$ is faced up, $0$ is faced down).

Some words now on making a move. Being in a state $s$, if we want to turn over a card $x$, we need to simply make a **xor** operation $s \wedge 2^{x-1}$. What does it mean? How does it work?

Suppose our state $s$ is equal to $0011_b = 3$, what means we have cards $1$ and $2$ faced down, and cards $3$ and $4$ faced up. If we want to turn over card $4$, our further state would be $1011_b = 11$.

In [2]:
#this is equivalent to Python xor operation:
print(3 ^ 8) #8 = pow(2,3)

11


The same holds if we want to make a card faced up, being already faced down. Suppose our state is again equal to $0011_b=3$ and we want to turn over the card $1$. Our further state would be $0010_b = 2$.

In [4]:
print(3 ^ 1) #1 = pow(2,0)

2


Of course, each state has a unique representation. From now on, we will use the binary notation while describing given states. So, talking about the state $1,2$ cards faced down, $3,4$ cards faced up, we will simply say state $3$.

Therefore, we can create the system of equations which solution will provide us with the answer.

Let $t_n$ denotes the expected number of moves to achieve the final state $2^{2N}-1$, (the absorbing one, in terms of the Markov chains), assuming we always follow the optimal strategy.

Then, the following system of equations must be satisfied:

$t_0 = 1 + \frac{1}{N^2} (\sum_{(x,y) \in R} min \{t_{0 \wedge 2^{x-1}}, t_{0 \wedge 2^{y-1}}, t_{0 \wedge 2^{x+y-1}} \})$;

$t_1 = 1 + \frac{1}{N^2} (\sum_{(x,y) \in R} min \{t_{1 \wedge 2^{x-1}}, t_{1 \wedge 2^{y-1}}, t_{1 \wedge 2^{x+y-1}} \})$;

$\ldots$

$t_{2^{2N}-2} = 1 + \frac{1}{N^2} (\sum_{(x,y) \in R} min \{t_{(2^{2N}-2) \wedge 2^{x-1}}, t_{(2^{2N}-2) \wedge 2^{y-1}}, t_{(2^{2N}-2) \wedge 2^{x+y-1}} \})$;

$t_{2^{2N}-1} = 0$.

Some comments:
    
$R$ denotes the rolls space. For $N=2$ that would be the set $\{(1,1),(1,2),(2,1),(2,2)\}$.

For the final state $2^{2N}-1$ the expected value is of course equal to $0$ - since we do not do any other moves.

For some other state $s$, we have $N^2$ possible rolls (that's why we have $\frac{1}{N^2}$, they are equally probable). For each roll in this state, we have at most three possible options - to choose one state from the set $\{t_{s \wedge 2^{x-1}}, t_{s \wedge 2^{y-1}}, t_{s \wedge 2^{x+y-1}}\}$.

We want to choose the option which leads to the state with the lowest expected number of moves to win - that's why we have $min \{t_{s \wedge 2^{x-1}}, t_{s \wedge 2^{y-1}}, t_{s \wedge 2^{x+y-1}}\}$.

While moving from a state $s$ to some state $s'$ (with some probability and decision), the expected number of moves to win for the state $s$ is equal to the expected number of moves to win for the state $s'$ plus one - since we need to do one move to reach the state $s'$.

We can express this system of equations in terms of function $f : \mathbb{R}^{2^{2N}} \rightarrow \mathbb{R}^{2^{2N}}$, defined below:

\begin{align}
f(t_0, t_1, \ldots, t_{2^{2N}-1}) = (&1 + \frac{1}{N^2} (\sum_{(x,y) \in R} min \{t_{0 \wedge 2^{x-1}}, t_{0 \wedge 2^{y-1}}, t_{0 \wedge 2^{x+y-1}} \}), \\ &1 + \frac{1}{N^2} (\sum_{(x,y) \in R} min \{t_{1 \wedge 2^{x-1}}, t_{1 \wedge 2^{y-1}}, t_{1 \wedge 2^{x+y-1}} \}), \\ &\ldots, \\ &1 + \frac{1}{N^2} (\sum_{(x,y) \in R} min \{t_{(2^{2N}-2) \wedge 2^{x-1}}, t_{(2^{2N}-2) \wedge 2^{y-1}}, t_{(2^{2N}-2) \wedge 2^{x+y-1}} \}), \\ &0).
\end{align}

This equation will be solved using the iterative point method. Starting from some arbitrary point $\hat{t} = (\hat{t}_{0}, \hat{t}_{1}, \ldots, \hat{t}_{2^{2N}-2}, \hat{t}_{2^{2N}-1})$, we are looking for the solution, creating iterations $\hat{t}, f(\hat{t}), f(f(\hat{t})), \ldots$, till some tolerance condition is satisfied.

Suppose $\hat{t} = (M, M, \ldots, M)$, where $M=10$. Let us compute the expected value for $N=5$.

In [7]:
from time import perf_counter

#Initial settings.
st = perf_counter()
M = 10
N = 5
t = [M]*pow(2,2*N)
tolerance = pow(10,-10)

#Filling out the rolls space.
rolls = []
for x in range(1, N+1):
    for y in range(1, N+1):
        rolls.append((x,y))

#Function checking whether to terminate the iteration process.
#It is terminated if the absolute value of the difference 
#between the current and previous expected value for each state is smaller than the predefined tolerance.
def is_terminated(t, new_t):
    for i in range(len(t)):
        if abs(t[i] - new_t[i]) > tolerance:
            return False
    return True

#Function f - defined as above in the notebook.
def f(t):
    new_t = []
    for i in range(len(t)-1):
        sm = 0
        for roll in rolls:
            x, y = roll
            sm += min(t[i ^ pow(2,x-1)], t[i ^ pow(2,y-1)], t[i ^ pow(2,x+y-1)])
        sm /= pow(N,2)
        new_t.append(1+sm)
    new_t.append(0)
    return new_t

#Iterating process, finished once is_terminated returns True.
ind = 0
while 1:
    new_t = f(t)
    if is_terminated(t, new_t): break
    if ind % 100 == 0: print(ind, sum(t))
    t = new_t
    ind += 1

print(ind, 'iterations needed.')
print('Result:', round(t[0],6))
print('Runtime:', round(perf_counter()-st, 2), 's.')

0 10240
100 24242.43966167763
200 24387.367262398937
300 24389.807502622574
400 24389.848669104606
500 24389.84936359602
569 iterations needed.
Result: 34.69856
Runtime: 19.13 s.
